In [1]:
pip install colorthief openai langchain langchain_community langchain_openai cairosvg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
import random
from PIL import Image, ImageDraw, ImageFont, ImageOps, ImageFilter
from textwrap import wrap
from colorthief import ColorThief
import io
import cairosvg
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# OpenAI function to highlight the important word
def text_highlight(text):
    system_prompt = """Identify the most important word or phrase in the following title. Consider terms related to technology, science, and innovation as important. Return only the word or phrase."""
    human_prompt = f"{text}"

    chat1 = ChatOpenAI(
        temperature=0.5,
        max_tokens=100,
        openai_api_key="sk-proj-1qcA4Zqo31qKge7pcbRgT3BlbkFJkQxYp0F6oRTCqeZxQPvh",
        model_name="gpt-4"
    )

    prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", human_prompt)])
    chain1 = prompt | chat1
    output1 = chain1.invoke({"text": text})

    subject_word = output1.content.strip()
    print(f"OpenAI identified important word: {subject_word}")
    return subject_word

def highlight_important_word(title):
    important_word = text_highlight(title)
    modified_title = title.replace(important_word, f"{important_word}")
    return modified_title, important_word

# Ensure title does not exceed 4 words
def limit_title_to_four_words(title):
    words = title.split()
    return ' '.join(words[:4])

# Ensure title does not exceed 25 characters and split into two lines if necessary
def format_title(title, max_length=25):
    title = limit_title_to_four_words(title)
    words = title.split()
    if len(title) > max_length:
        midpoint = len(words) // 2
        line1 = ' '.join(words[:midpoint])
        line2 = ' '.join(words[midpoint:])
        return f"{line1}\n{line2}"
    return title

# Function to get the dominant color of an image
def get_dominant_color(image):
    with io.BytesIO() as output:
        image.save(output, format="PNG")
        output.seek(0)
        color_thief = ColorThief(output)
        return color_thief.get_color(quality=1)

# Function to generate gradient color
def generate_gradient_color(color1, color2, width, height):
    base = Image.new('RGB', (width, height), color1)
    top = Image.new('RGB', (width, height), color2)
    mask = Image.new('L', (width, height))
    mask_data = [int(255 * (x / width)) for y in range(height) for x in range(width)]
    mask.putdata(mask_data)
    base.paste(top, (0, 0), mask)
    return base.filter(ImageFilter.SMOOTH)

# Function to determine text color based on background color with a set of colors
def get_contrasting_text_color(background_color):
    # Calculate the brightness of the background color
    brightness = (background_color[0] * 299 + background_color[1] * 587 + background_color[2] * 114) / 1000

    # Set of colors to use for text
    text_colors = [
        (0, 0, 0), (255, 255, 255), (255, 69, 0), (255, 140, 0), (255, 215, 0),
        (0, 128, 128), (0, 0, 255), (75, 0, 130), (238, 130, 238), (0, 128, 0)
    ]

    # Choose a text color that contrasts well with the background
    if brightness < 128:
        # For dark backgrounds, choose a light text color
        contrasting_colors = [color for color in text_colors if (color[0] * 299 + color[1] * 587 + color[2] * 114) / 1000 > 128]
    else:
        # For light backgrounds, choose a dark text color
        contrasting_colors = [color for color in text_colors if (color[0] * 299 + color[1] * 587 + color[2] * 114) / 1000 <= 128]

    # Randomly select a color from the set of contrasting colors
    return random.choice(contrasting_colors)

# Function to adjust text color if it matches the background
def adjust_text_color(background_color, text_color):
    if background_color == text_color:
        return get_complementary_color(background_color)
    return text_color

# List of contrasting colors
contrasting_colors = [
    (0, 51, 102), (51, 0, 102), (102, 0, 51), (102, 51, 0),
    (51, 51, 153), (51, 0, 51), (0, 51, 0), (0, 51, 51),
    (0, 0, 102), (51, 51, 51), (51, 51, 0), (51, 102, 0),
    (255, 69, 0), (255, 140, 0), (255, 215, 0), (0, 128, 128)
]

# Function to calculate complementary color
def get_complementary_color(color):
    return (255 - color[0], 255 - color[1], 255 - color[2])

# Function to get a random contrasting color
def get_random_contrasting_color(exclude_colors=None):
    exclude_colors = exclude_colors or []
    colors = [color for color in contrasting_colors if color not in exclude_colors]
    return random.choice(colors)

# Function to sample the background color at multiple points
def sample_background_color(image, positions, area_size=10):
    colors = []
    for pos in positions:
        x, y = pos
        for i in range(-area_size // 2, area_size // 2):
            for j in range(-area_size // 2, area_size // 2):
                try:
                    colors.append(image.getpixel((x + i, y + j)))
                except IndexError:
                    continue
    avg_color = tuple(sum(c) // len(c) for c in zip(*colors))
    return avg_color

# Function to generate random circles
def generate_random_circles(num_circles, width, height, bounding_boxes):
    circles = []
    for _ in range(num_circles):
        while True:
            x = random.randint(0, width // 4) if random.choice([True, False]) else random.randint(3 * width // 4, width)
            y = random.randint(0, height // 4) if random.choice([True, False]) else random.randint(3 * height // 4, height)
            radius = random.randint(50, 150)
            circle_box = (x - radius, y - radius, x + radius, y + radius)
            if not any(is_overlap(circle_box, box) for box in bounding_boxes):
                circles.append((x, y, radius, (255, 255, 255, 64)))
                break
    return circles

# Function to generate random circles on the product
def generate_random_circles_on_product(num_circles, width, height, product_box, bounding_boxes):
    circles = []
    x1, y1, x2, y2 = product_box
    product_width = x2 - x1
    product_height = x2 - x1

    for _ in range(num_circles):
        for _ in range(100):  # Limit the number of attempts to place a circle
            x = random.randint(x1, x2)
            y = random.randint(y1, y2)
            radius = random.randint(50, 150)
            circle_box = (x - radius, y - radius, x + radius, y + radius)
            if not any(is_overlap(circle_box, box) for box in bounding_boxes):
                circles.append((x, y, radius, (255, 255, 255, 64)))
                break
    return circles

# Function to wrap text based on newline characters and width
def wrap_text(text, max_width, draw, font):
    lines = []
    for line in text.split('\n'):
        lines.extend(wrap(line, width=max_width, break_long_words=False))
    return lines

# Function to check if two bounding boxes overlap
def is_overlap(box1, box2):
    return not (box1[2] < box2[0] or box1[0] > box2[2] or box1[3] < box2[1] or box1[1] > box2[3])

# Function to adjust font size based on available space
def adjust_font_size_based_on_space(draw, text, font_path, max_width, max_height, additional_size=0):
    min_font_size = 28
    max_font_size = 40
    font_size = min_font_size + additional_size

    while font_size <= max_font_size + additional_size:
        font = ImageFont.truetype(font_path, font_size)
        text_width, text_height = draw.textsize(text, font=font)
        if text_width <= max_width and text_height <= max_height:
            font_size += 1
        else:
            break

    return ImageFont.truetype(font_path, font_size - 1 if font_size > min_font_size else font_size)

def draw_element(draw, key, position, size, text=None, font=None, element_type="text", image_path=None, cta_shape="rectangle", text_color=(0, 0, 0), background_color=(255, 255, 255), highlight_word=None, highlight_font=None, highlight_color=None, mobile_icon=None):
    x = int(position[0] * width)
    y = int(position[1] * height)

    if element_type == "text":
        max_width = int(size[0] * width) if size else width
        wrapped_text = wrap_text(text, 32, draw, font)
        max_line_width = max(draw.textbbox((0, 0), line, font=font)[2] for line in wrapped_text)
        line_height = font.getbbox('A')[3]
        gap = int(line_height * 0.4)
        total_height = (line_height + gap) * len(wrapped_text) - gap

        if key == "cta":
            padding = 10
            high_res_width = max_line_width + padding * 2
            high_res_height = total_height + padding * 2

            # Increase resolution by 8x for anti-aliasing
            high_res_button = Image.new('RGBA', (high_res_width * 8, high_res_height * 8), (0, 0, 0, 0))
            high_res_draw = ImageDraw.Draw(high_res_button)

            if cta_shape == "rounded":
                radius = 15 * 8
                high_res_draw.rounded_rectangle(
                    [(0, 0), (high_res_width * 8, high_res_height * 8)],
                    radius=radius,
                    fill=cta_background_color
                )
            else:
                high_res_draw.rectangle(
                    [(0, 0), (high_res_width * 8, high_res_height * 8)],
                    fill=cta_background_color
                )

            # Smooth the edges of the CTA button by downscaling
            high_res_button = high_res_button.resize((high_res_width, high_res_height), Image.LANCZOS)

            image.paste(high_res_button, (x - padding, y - padding), high_res_button)

        if key == "desc_first_word":
            first_sentence_end = text.find('\n') if '\n' in text else len(text)
            first_sentence = text[:first_sentence_end]
            remaining_text = text[first_sentence_end:].strip()

            # Calculate a contrasting color for the first sentence
            first_sentence_color = get_contrasting_text_color(background_color)

            # Increase the size of the first sentence
            first_sentence_font = ImageFont.truetype(font.path, font.size + 6)  # Increase size by 6 points
            first_sentence_bbox = draw.textbbox((0, 0), first_sentence, font=first_sentence_font)
            first_sentence_width = first_sentence_bbox[2] - first_sentence_bbox[0]
            first_sentence_height = first_sentence_bbox[3] - first_sentence_bbox[1]

            # Draw the first sentence
            text_x = x
            text_y = y
            draw.text((text_x, text_y), first_sentence, first_sentence_color, first_sentence_font)

            wrapped_text = wrap_text(remaining_text, 30, draw, font)
            desc_y = text_y + first_sentence_height + gap + 30  # Increase gap between first sentence and remaining text
            for i, line in enumerate(wrapped_text):
                text_x = x
                text_y = desc_y + i * (line_height + gap)
                draw.text((text_x, text_y), line, text_color, font)

            return (x, y, x + first_sentence_width, text_y + line_height)

        elif key == "contact":
            icon_size = int(line_height * 1.5)  # Adjust the size of the mobile icon
            if mobile_icon is not None:
                mobile_icon_resized = mobile_icon.resize((icon_size, icon_size), Image.LANCZOS)
                image.paste(mobile_icon_resized, (x, y), mobile_icon_resized)
                x += icon_size + 5  # Add some padding after the icon

            for i, line in enumerate(wrapped_text):
                text_x = x
                text_y = y + i * (line_height + gap)
                draw.text((text_x, text_y), line, fill=text_color, font=font)  # Change contact text color to highlight color

            return (x - icon_size - 5, y, x + max_line_width, y + total_height)

        else:
            for i, line in enumerate(wrapped_text):
                text_x = x
                text_y = y + i * (line_height + gap)
                # Highlight the important word with the same size as the title
                if highlight_word and highlight_word in line:
                    pre_text, post_text = line.split(highlight_word, 1)
                    draw.text((text_x, text_y), pre_text, text_color, font)
                    highlight_width = draw.textbbox((0, 0), highlight_word, font=font)[2]
                    draw.text((text_x + draw.textbbox((0, 0), pre_text, font=font)[2] + 5, text_y), highlight_word, highlight_color, font)
                    draw.text((text_x + draw.textbbox((0, 0), pre_text + " " + highlight_word, font=font)[2] + 5, text_y), post_text, text_color, font)
                else:
                    draw.text((text_x, text_y), line, text_color, font)

            return (x, y, x + max_line_width, y + total_height)

    elif element_type == "image" and image_path:
        try:
            if image_path.lower().endswith('.svg'):
                png_image = cairosvg.svg2png(url=image_path)
                img = Image.open(io.BytesIO(png_image)).convert("RGBA")
            else:
                img = Image.open(image_path).convert("RGBA")
        except OSError:
            print(f"Error opening image: {image_path}")
            return None

        if "product" in key:
            x = int(position[0] * width)
            y = int(position[1] * height)
            max_width = int(size[0] * width)
            max_height = int(size[1] * height)

            # Ensure the product image fits within the available space without circles or other elements
            img = img.crop(img.getbbox())
            img.thumbnail((max_width, max_height), Image.LANCZOS)
            image.paste(img, (x, y), img)
            return (x, y, x + img.width, y + img.height)
        elif "logo" in key:
            logo_width = int(size[0] * width)
            logo_height = int(size[1] * height)

            # Ensure the logo image fits within the available space without blending
            img = img.crop(img.getbbox())
            img.thumbnail((logo_width, logo_height), Image.LANCZOS)

            x = int(position[0] * width)
            y = int(position[1] * height)

            # Create a temporary image for the logo with a transparent background
            temp_logo = Image.new('RGBA', (logo_width, logo_height), (0, 0, 0, 0))
            temp_logo.paste(img, (0, 0), img)

            image.paste(temp_logo, (x, y), temp_logo)
            return (x, y, x + temp_logo.width, y + temp_logo.height)

# Function to blend circles
def blend_circles(background, circles):
    blended_image = background.copy()
    draw = ImageDraw.Draw(blended_image, "RGBA")
    for (x, y, radius, color) in circles:
        circle = Image.new('RGBA', (radius * 2 * 8, radius * 2 * 8), (0, 0, 0, 0))  # Increase size by 8x for anti-aliasing
        circle_draw = ImageDraw.Draw(circle)
        circle_draw.ellipse((0, 0, radius * 2 * 8, radius * 2 * 8), fill=color)
        circle = circle.resize((radius * 2, radius * 2), Image.LANCZOS)  # Downscale
        blended_image.paste(circle, (x - radius, y - radius), circle)
    return blended_image

font_path_heading = "ARIALBD.TTF"
font_path_desc = "arial.ttf"
font_path_cta = "ARIALBD.TTF"
font_path_contact = "arial.ttf"
mobile_icon_path = "icon.png"  # Replace with the actual path to the mobile icon

heading = "Stay Fit Stay Healthy"
desc = "*Health Benefits*\n- Support digestive health\n- Support immune health\n- Restore good bacteria in the intestine\n- Improve digestion"
cta = "Buy Now"
contact = "123-456-7890"
logo_path = "Logo-var2.png"  # Replace with actual path
product_path = "IMG_0491 copy.png"  # Replace with actual path

heading = format_title(heading)  # Ensure heading is limited to 4 words and formatted
highlighted_heading, important_word = highlight_important_word(heading)
highlight_font = ImageFont.truetype(font_path_heading, 40)  # Define the highlight font

# Load the mobile icon
try:
    mobile_icon = Image.open(mobile_icon_path).convert("RGBA")
except OSError:
    print(f"Error opening mobile icon: {mobile_icon_path}")
    mobile_icon = None

layouts = [
    {
        "logo": [(0.45, 0.85, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.1, 0.75, 0.6, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.03, 0.3, 0.8, 0.15), desc, "text", font_path_desc],
        "cta": [(0.2, 0.2, 0.2, 0.1), cta, "text", font_path_cta],
        "contact": [(0.03, 0.05, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.55, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.04, 0.06, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.3, 0.06, 0.6, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.03, 0.3, 0.8, 0.15), desc, "text", font_path_desc],
        "cta": [(0.2, 0.75, 0.2, 0.1), cta, "text", font_path_cta],
        "contact": [(0.1, 0.85, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.55, 0.3, 0.6, 0.6), product_path, "image"]
    },
    {
        "logo": [(0.7, 0.05, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.05, 0.06, 0.6, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.5, 0.3, 0.8, 0.15), desc, "text", font_path_desc],
        "cta": [(0.2, 0.9, 0.2, 0.1), cta, "text", font_path_cta],
        "contact": [(0.5, 0.8, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.03, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.05, 0.01, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.4, 0.05, 0.6, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.05, 0.3, 0.6, 0.2), desc, "text", font_path_desc],
        "cta": [(0.6, 0.9, 0.2, 0.1), cta, "text", font_path_cta],
        "contact": [(0.05, 0.8, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.60, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.3, 0.01, 0.25, 0.15), logo_path, "image"],
        "heading": [(0.42, 0.2, 0.7, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.5, 0.4, 0.8, 0.15), desc, "text", font_path_desc],
        "cta": [(0.1, 0.85, 0.3, 0.1), cta, "text", font_path_cta],
        "contact": [(0.5, 0.85, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.03, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.02, 0.80, 0.25, 0.15), logo_path, "image"],
        "heading": [(0.4, 0.05, 0.7, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.03, 0.3, 0.8, 0.15), desc, "text", font_path_desc],
        "cta": [(0.6, 0.85, 0.3, 0.1), cta, "text", font_path_cta],
        "contact": [(0.02, 0.05, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.60, 0.3, 0.4, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.75, 0.05, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.2, 0.06, 0.6, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.5, 0.3, 0.8, 0.15), desc, "text", font_path_desc],
        "cta": [(0.1, 0.85, 0.3, 0.1), cta, "text", font_path_cta],
        "contact": [(0.5, 0.77, 0.3, 0.1), contact, "text", font_path_contact],
        "product": [(0.05, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.03, 0.85, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.03, 0.15,  0.6, 0.1), heading, "text", font_path_heading, 2],
        "desc_first_word": [(0.5, 0.3, 0.8, 0.15), desc, "text", font_path_desc],
        "cta": [(0.6, 0.82, 0.3, 0.1), cta, "text", font_path_cta],
        "contact": [(0.5, 0.05, 0.2, 0.15), contact, "text", font_path_contact],
        "product": [(0.03, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.65, 0.02, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.03, 0.05,  0.6, 0.1), heading, "text", font_path_heading, 2],
        "desc_first_word": [(0.03, 0.3, 0.4, 0.4), desc, "text", font_path_desc],
        "cta": [(0.6, 0.85, 0.3, 0.1), cta, "text", font_path_cta],
        "contact": [(0.03, 0.85, 0.3, 0.1), contact, "text", font_path_contact],
        "product": [(0.55, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.65, 0.02, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.03, 0.05,  0.6, 0.1), heading, "text", font_path_heading, 2],
        "desc_first_word": [(0.03, 0.3, 0.4, 0.4), desc, "text", font_path_desc],
        "cta": [(0.6, 0.85, 0.3, 0.1), cta, "text", font_path_cta],
        "contact": [(0.03, 0.85, 0.3, 0.1), contact, "text", font_path_contact],
        "product": [(0.55, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.05, 0.1, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.3, 0.05, 0.7, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.05, 0.25, 0.7, 0.2), desc, "text", font_path_desc],
        "cta": [(0.6, 0.85, 0.2, 0.1), cta, "text", font_path_cta],
        "contact": [(0.05, 0.8, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.55, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.3, 0.05, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.05, 0.2, 0.7, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.05, 0.35, 0.7, 0.2), desc, "text", font_path_desc],
        "cta": [(0.6, 0.85, 0.2, 0.1), cta, "text", font_path_cta],
        "contact": [(0.05, 0.8, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.55, 0.3, 0.5, 0.5), product_path, "image"]
    },
    {
        "logo": [(0.7, 0.1, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.1, 0.1, 0.7, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.05, 0.25, 0.7, 0.2), desc, "text", font_path_desc],
        "cta": [(0.6, 0.85, 0.2, 0.1), cta, "text", font_path_cta],
        "contact": [(0.05, 0.8, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.55, 0.3, 0.5, 0.5), product_path, "image"]
    } ,
    {
        "logo": [(0.75, 0.01, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.2, 0.15, 0.7, 0.1), heading, "text", font_path_heading],
        "desc_first_word": [(0.05, 0.40, 0.8, 0.3), desc, "text", font_path_desc],
        "cta": [(0.66, 0.3, 0.2, 0.1), cta, "text", font_path_cta],
        "contact": [(0.03, 0.02, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.55, 0.4, 0.6, 0.6), product_path, "image"]
    },
    {
        "logo": [(0.75, 0.01, 0.2, 0.15), logo_path, "image"],
        "heading": [(0.05, 0.13, 0.8, 0.2), heading, "text", font_path_heading],
        "desc_first_word": [(0.5, 0.35, 0.8, 0.1), desc, "text", font_path_desc],
        "cta": [(0.15, 0.92, 0.2, 0.1), cta, "text", font_path_cta],
        "contact": [(0.03, 0.01, 0.2, 0.1), contact, "text", font_path_contact],
        "product": [(0.05, 0.3, 0.6, 0.6), product_path, "image"]
    }
]

grid_width = 50
grid_height = 50

def generate_grid(width, height):
    grid_cells = []
    for x in range(0, width, grid_width):
        for y in range(0, height, grid_height):
            grid_cells.append((x, y, x + grid_width, y + height))
    return grid_cells

def find_empty_spaces(bounding_boxes, width, height):
    grid_cells = generate_grid(width, height)
    empty_spaces = []
    for cell in grid_cells:
        cell_empty = True
        for box in bounding_boxes:
            if is_overlap(cell, box):
                cell_empty = False
                break
        if cell_empty:
            empty_spaces.append(cell)
    return empty_spaces

def expand_element(bounding_boxes, element_box, max_width, max_height):
    left_expand = right_expand = up_expand = down_expand = 0
    while all(not is_overlap(element_box, box) for box in bounding_boxes) and element_box[0] > 0 and element_box[2] < max_width and element_box[1] > 0 and element_box[3] < max_height:
        left_expand += 1
        right_expand += 1
        up_expand += 1
        down_expand += 1
        element_box = (element_box[0] - left_expand, element_box[1] - up_expand, element_box[2] + right_expand, element_box[3] + down_expand)
    return element_box

# Example of how to use this function when drawing the heading
for i, elements in enumerate(layouts):
    width, height = 800, 600
    color1 = tuple(random.randint(0, 255) for _ in range(3))
    color2 = tuple(random.randint(0, 255) for _ in range(3))
    while color1 == (255, 105, 180) or color1 == (0, 255, 0):  # Avoid pink and green background
        color1 = tuple(random.randint(0, 255) for _ in range(3))
    while color2 == (255, 105, 180) or color2 == (0, 255, 0):  # Avoid pink and green background
        color2 = tuple(random.randint(0, 255) for _ in range(3))
    background = generate_gradient_color(color1, color2, width, height)
    image = Image.new("RGBA", (width, height))
    image.paste(background, (0, 0))
    draw = ImageDraw.Draw(image)
    bounding_boxes = []

    dominant_color = get_dominant_color(image)
    highlight_color = get_random_contrasting_color([dominant_color])
    heading_color = get_random_contrasting_color([highlight_color])
    desc_color = get_random_contrasting_color([heading_color, highlight_color])

    shapes = ["rounded", "rectangle"]
    random.shuffle(shapes)

    # Draw text elements first to get bounding boxes
    for key, value in elements.items():
        pos, text, elem_type, *font = value
        pos_x, pos_y = int(pos[0] * width), int(pos[1] * height)
        pos_w, pos_h = int(pos[2] * width), int(pos[3] * height)

        sampled_background_color = sample_background_color(image, [(pos_x + pos_w // 2, pos_y + pos_h // 2)])

        print(f"Element: {key}, Position: {pos}, Sampled Background Color: {sampled_background_color}")

        if elem_type == "text":
            max_width = int(pos[2] * width)
            max_height = int(pos[3] * height)
            if key == "heading":
                additional_size = font[1] if len(font) > 1 else 0
                adjusted_font = adjust_font_size_based_on_space(draw, text, font[0], max_width, max_height, additional_size)
                text_color = adjust_text_color(sampled_background_color, heading_color)
                print(f"Heading color: {text_color}")
            elif key == "desc_first_word":
                adjusted_font = ImageFont.truetype(font[0], 26)  # Default font size for other text elements
                text_color = adjust_text_color(sampled_background_color, desc_color)
                print(f"Description color: {text_color}")
            elif key == "contact":
                adjusted_font = ImageFont.truetype(font[0], 28)  # Default font size for other text elements
                text_color = get_contrasting_text_color(sampled_background_color)
                print(f"Contact color: {text_color}")
            else:
                adjusted_font = ImageFont.truetype(font[0], 26)  # Default font size for other text elements
                text_color = get_contrasting_text_color(sampled_background_color)
                print(f"Other text color: {text_color}")

            wrapped_text = wrap_text(text, 30, draw, adjusted_font)[:6]
            if key == "desc_first_word":
                box = draw_element(draw, key, pos[:2], pos[2:], '\n'.join(wrapped_text), adjusted_font, element_type=elem_type, cta_shape=shapes[0], text_color=text_color, background_color=sampled_background_color)
            elif key == "cta":
                cta_background_color = get_complementary_color(sampled_background_color)  # Set CTA background color to complement the background color at its position
                cta_text_color = get_contrasting_text_color(cta_background_color)
                print(f"CTA button color: {cta_background_color}, CTA text color: {cta_text_color}")
                box = draw_element(draw, key, pos[:2], pos[2:], '\n'.join(wrapped_text), adjusted_font, element_type=elem_type, cta_shape=shapes[1], text_color=cta_text_color, background_color=cta_background_color)
            elif key == "contact":
                box = draw_element(draw, key, pos[:2], pos[2:], '\n'.join(wrapped_text), adjusted_font, element_type=elem_type, cta_shape="rectangle", text_color=text_color, mobile_icon=mobile_icon)
            else:
                box = draw_element(draw, key, pos[:2], pos[2:], '\n'.join(wrapped_text), adjusted_font, element_type=elem_type, cta_shape="rounded", text_color=text_color, highlight_word=important_word, highlight_font=adjusted_font, highlight_color=highlight_color)
            bounding_boxes.append(box)
        else:
            try:
                if text.lower().endswith('.svg'):
                    png_image = cairosvg.svg2png(url=text)
                    img = Image.open(io.BytesIO(png_image)).convert("RGBA")
                else:
                    img = Image.open(text).convert("RGBA")
            except OSError:
                print(f"Error opening image: {text}")
                continue

            max_width = int(pos[2] * width)
            max_height = int(pos[3] * height)
            img.thumbnail((max_width, max_height), Image.LANCZOS)
            box = draw_element(draw, key, pos[:2], (img.width / width, img.height / height), None, None, element_type=elem_type, image_path=text)
            bounding_boxes.append(box)

    # Generate random circles with one or two on the product image
    product_box = [int(pos[0] * width), int(pos[1] * height), int(pos[0] * width + pos[2] * width), int(pos[1] * height + pos[3] * height)]
    random_circles_on_product = generate_random_circles_on_product(2, width, height, product_box, bounding_boxes)
    random_circles = generate_random_circles(3, width, height, bounding_boxes)  # Limit to 5 shapes total

    all_circles = random_circles_on_product + random_circles
    blended_background = blend_circles(background, all_circles)
    image.paste(blended_background, (0, 0))

    for key, value in elements.items():
        pos, text, elem_type, *font = value
        pos_x, pos_y = int(pos[0] * width), int(pos[1] * height)
        pos_w, pos_h = int(pos[2] * width), int(pos[3] * height)

        sampled_background_color = sample_background_color(image, [(pos_x + pos_w // 2, pos_y + pos_h // 2)])

        print(f"Element: {key}, Position: {pos}, Sampled Background Color: {sampled_background_color}")

        if elem_type == "text":
            max_width = int(pos[2] * width)
            max_height = int(pos[3] * height)
            if key == "heading":
                additional_size = font[1] if len(font) > 1 else 0
                adjusted_font = adjust_font_size_based_on_space(draw, text, font[0], max_width, max_height, additional_size)
                text_color = adjust_text_color(sampled_background_color, heading_color)
                print(f"Heading color: {text_color}")
            elif key == "desc_first_word":
                adjusted_font = ImageFont.truetype(font[0], 26)  # Default font size for other text elements
                text_color = adjust_text_color(sampled_background_color, desc_color)
                print(f"Description color: {text_color}")
            elif key == "contact":
                adjusted_font = ImageFont.truetype(font[0], 30)  # Default font size for other text elements
                text_color = get_contrasting_text_color(sampled_background_color)
                print(f"Contact color: {text_color}")
            else:
                adjusted_font = ImageFont.truetype(font[0], 24)  # Default font size for other text elements
                text_color = get_contrasting_text_color(sampled_background_color)
                print(f"Other text color: {text_color}")

            wrapped_text = wrap_text(text, 30, draw, adjusted_font)[:6]
            if key == "desc_first_word":
                draw_element(draw, key, pos[:2], pos[2:], '\n'.join(wrapped_text), adjusted_font, element_type=elem_type, cta_shape=shapes[0], text_color=text_color, background_color=sampled_background_color)
            elif key == "cta":
                cta_background_color = get_complementary_color(sampled_background_color)  # Set CTA background color to complement the background color at its position
                cta_text_color = get_contrasting_text_color(cta_background_color)
                print(f"CTA button color: {cta_background_color}, CTA text color: {cta_text_color}")
                draw_element(draw, key, pos[:2], pos[2:], '\n'.join(wrapped_text), adjusted_font, element_type=elem_type, cta_shape=shapes[1], text_color=cta_text_color, background_color=cta_background_color)
            elif key == "contact":
                draw_element(draw, key, pos[:2], pos[2:], '\n'.join(wrapped_text), adjusted_font, element_type=elem_type, cta_shape="rectangle", text_color=text_color, mobile_icon=mobile_icon)
            else:
                highlight_color = get_contrasting_text_color(dominant_color)
                draw_element(draw, key, pos[:2], pos[2:], '\n'.join(wrapped_text), adjusted_font, element_type=elem_type, cta_shape="rounded", text_color=text_color, highlight_word=important_word, highlight_font=adjusted_font, highlight_color=highlight_color)
        else:
            try:
                if text.lower().endswith('.svg'):
                    png_image = cairosvg.svg2png(url=text)
                    img = Image.open(io.BytesIO(png_image)).convert("RGBA")
                else:
                    img = Image.open(text).convert("RGBA")
            except OSError:
                print(f"Error opening image: {text}")
                continue

            max_width = int(pos[2] * width)
            max_height = int(pos[3] * height)
            img.thumbnail((max_width, max_height), Image.LANCZOS)
            draw_element(draw, key, pos[:2], (img.width / width, img.height / height), None, None, element_type=elem_type, image_path=text)

    # Adjust title position if it overlaps with other elements
    for j, box in enumerate(bounding_boxes):
        if key == "heading" and is_overlap(expanded_box, box):
            y_shift = box[3] - expanded_box[1] + 10  # Shift downward by the overlapping amount plus some padding
            expanded_box = (expanded_box[0], expanded_box[1] + y_shift, expanded_box[2], expanded_box[3] + y_shift)
            draw_element(draw, key, pos[:2], pos[2:], '\n'.join(wrapped_text), adjusted_font, element_type=elem_type, cta_shape=shapes[0], text_color=heading_color)

    image.save(f"ad_template_{i+1}.png", quality=95)
    empty_spaces = find_empty_spaces(bounding_boxes, width, height)
    print(f"Empty spaces in layout {i+1}: {empty_spaces}")

image.show()


OpenAI identified important word: Fit
Element: logo, Position: (0.45, 0.85, 0.2, 0.15), Sampled Background Color: (134, 127, 174, 255)
Element: heading, Position: (0.1, 0.75, 0.6, 0.1), Sampled Background Color: (103, 110, 177, 255)
Heading color: (102, 0, 51)
Element: desc_first_word, Position: (0.03, 0.3, 0.8, 0.15), Sampled Background Color: (110, 113, 176, 255)
Description color: (255, 215, 0)


<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10

Element: cta, Position: (0.2, 0.2, 0.2, 0.1), Sampled Background Color: (83, 98, 178, 255)
Other text color: (255, 255, 255)
CTA button color: (172, 157, 77), CTA text color: (0, 128, 128)
Element: contact, Position: (0.03, 0.05, 0.2, 0.1), Sampled Background Color: (49, 79, 181, 255)
Contact color: (255, 215, 0)
Element: product, Position: (0.55, 0.3, 0.5, 0.5), Sampled Background Color: (184, 155, 170, 255)
Element: logo, Position: (0.45, 0.85, 0.2, 0.15), Sampled Background Color: (134, 127, 174, 255)
Element: heading, Position: (0.1, 0.75, 0.6, 0.1), Sampled Background Color: (103, 110, 177, 255)
Heading color: (102, 0, 51)
Element: desc_first_word, Position: (0.03, 0.3, 0.8, 0.15), Sampled Background Color: (110, 113, 176, 255)
Description color: (255, 215, 0)
Element: cta, Position: (0.2, 0.2, 0.2, 0.1), Sampled Background Color: (83, 98, 178, 255)
Other text color: (255, 255, 255)
CTA button color: (172, 157, 77), CTA text color: (0, 0, 0)


<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10

Element: contact, Position: (0.03, 0.05, 0.2, 0.1), Sampled Background Color: (49, 79, 181, 255)
Contact color: (255, 255, 255)
Element: product, Position: (0.55, 0.3, 0.5, 0.5), Sampled Background Color: (184, 155, 170, 255)
Empty spaces in layout 1: [(0, 400, 50, 1000), (0, 450, 50, 1050), (0, 500, 50, 1100), (0, 550, 50, 1150), (50, 500, 100, 1100), (50, 550, 100, 1150), (100, 500, 150, 1100), (100, 550, 150, 1150), (150, 500, 200, 1100), (150, 550, 200, 1150), (200, 500, 250, 1100), (200, 550, 250, 1150), (250, 500, 300, 1100), (250, 550, 300, 1150), (300, 500, 350, 1100), (300, 550, 350, 1150), (550, 500, 600, 1100), (550, 550, 600, 1150), (600, 500, 650, 1100), (600, 550, 650, 1150), (650, 500, 700, 1100), (650, 550, 700, 1150), (700, 500, 750, 1100), (700, 550, 750, 1150), (750, 0, 800, 600), (750, 50, 800, 650), (750, 100, 800, 700), (750, 150, 800, 750), (750, 200, 800, 800), (750, 250, 800, 850), (750, 300, 800, 900), (750, 350, 800, 950), (750, 400, 800, 1000), (750, 450, 80

...

Element: cta, Position: (0.66, 0.3, 0.2, 0.1), Sampled Background Color: (63, 197, 163, 255)
Other text color: (0, 0, 255)
CTA button color: (192, 58, 92), CTA text color: (255, 215, 0)
Element: contact, Position: (0.03, 0.02, 0.2, 0.1), Sampled Background Color: (217, 66, 48, 255)
Contact color: (238, 130, 238)
Element: product, Position: (0.55, 0.4, 0.6, 0.6), Sampled Background Color: (41, 216, 180, 255)
Empty spaces in layout 14: [(0, 500, 50, 1100), (0, 550, 50, 1150), (50, 500, 100, 1100), (50, 550, 100, 1150), (100, 500, 150, 1100), (100, 550, 150, 1150), (150, 500, 200, 1100), (150, 550, 200, 1150), (200, 500, 250, 1100), (200, 550, 250, 1150), (250, 500, 300, 1100), (250, 550, 300, 1150), (300, 150, 350, 750), (300, 200, 350, 800), (300, 250, 350, 850), (300, 300, 350, 900), (300, 350, 350, 950), (300, 400, 350, 1000), (300, 450, 350, 1050), (300, 500, 350, 1100), (300, 550, 350, 1150), (350, 150, 400, 750), (350, 200, 400, 800), (350, 250, 400, 850), (350, 300, 400, 900), (35

<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10

Element: cta, Position: (0.15, 0.92, 0.2, 0.1), Sampled Background Color: (114, 158, 218, 255)
Other text color: (75, 0, 130)
CTA button color: (141, 97, 37), CTA text color: (255, 140, 0)
Element: contact, Position: (0.03, 0.01, 0.2, 0.1), Sampled Background Color: (96, 177, 229, 255)
Contact color: (0, 0, 0)
Element: product, Position: (0.05, 0.3, 0.6, 0.6), Sampled Background Color: (129, 144, 209, 255)
Element: logo, Position: (0.75, 0.01, 0.2, 0.15), Sampled Background Color: (202, 68, 163, 255)
Element: heading, Position: (0.05, 0.13, 0.8, 0.2), Sampled Background Color: (143, 128, 200, 255)
Heading color: (0, 0, 102)
Element: desc_first_word, Position: (0.5, 0.35, 0.8, 0.1), Sampled Background Color: (209, 61, 159, 255)
Description color: (102, 0, 51)


<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
<ipython-input-15-567ea09ead35>:180: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10

Element: cta, Position: (0.15, 0.92, 0.2, 0.1), Sampled Background Color: (114, 158, 218, 255)
Other text color: (0, 128, 128)
CTA button color: (141, 97, 37), CTA text color: (255, 255, 255)
Element: contact, Position: (0.03, 0.01, 0.2, 0.1), Sampled Background Color: (96, 177, 229, 255)
Contact color: (0, 128, 0)
Element: product, Position: (0.05, 0.3, 0.6, 0.6), Sampled Background Color: (129, 144, 209, 255)
Empty spaces in layout 15: [(0, 550, 50, 1150), (50, 550, 100, 1150), (250, 550, 300, 1150), (300, 550, 350, 1150), (350, 550, 400, 1150), (400, 450, 450, 1050), (400, 500, 450, 1100), (400, 550, 450, 1150), (450, 450, 500, 1050), (450, 500, 500, 1100), (450, 550, 500, 1150), (500, 450, 550, 1050), (500, 500, 550, 1100), (500, 550, 550, 1150), (550, 450, 600, 1050), (550, 500, 600, 1100), (550, 550, 600, 1150), (600, 450, 650, 1050), (600, 500, 650, 1100), (600, 550, 650, 1150), (650, 100, 700, 700), (650, 150, 700, 750), (650, 200, 700, 800), (650, 250, 700, 850), (650, 300, 70

# title color is not changing